In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
credits= pd.read_csv("E:/DataScienceProject/archive/tmdb_5000_credits.csv")

In [ ]:
movies=pd.read_csv("E:/DataScienceProject/tmdb_5000_movies.csv")


movies.head(5)

In [ ]:
movies

In [ ]:
credits

In [ ]:
credits.shape

In [ ]:
movies.shape

In [ ]:
movies= movies.merge( credits, on ='title')

In [ ]:
movies.shape

In [ ]:
#will recoomendate on 7 criteria 
#genres
#id
#keyword
#title
#overview
#cast
#crew
movies.columns


In [ ]:
movies= movies[['genres','id', 'keywords','title', 'overview','cast', 'crew']]

In [ ]:
movies

In [ ]:
movies.shape

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace =True)
movies.isnull().sum()

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
import ast
def convert(obj) :
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
        return l

In [ ]:
movies.head()

In [ ]:
movies

In [ ]:
movies.iloc[0].keywords

In [ ]:
movies['keywords']=movies['keywords'].apply(convert)

In [ ]:
movies

In [ ]:
movies

In [ ]:
movies['genres']=movies['genres'].apply(convert)

In [ ]:
movies

In [ ]:
movies

In [ ]:
import ast

def convert_cast(obj):
    l = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            l.append(i['name'])
            counter += 1
        else:
            break
    return l


In [ ]:
movies['cast']=movies['cast'].apply(convert_cast)

In [ ]:
movies

In [ ]:
import ast

def fetch_director(obj):
    l = []
    
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            l.append(i['name'])
            break
    return l

In [ ]:
movies['crew']=movies['crew'].apply(fetch_director)

In [ ]:
movies

In [ ]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [ ]:
def remove_spac(word):
    l=[]
    for i in word:
        l.append(i.replace(" ",""))
        return l



In [ ]:

movies['cast']=movies['cast'].apply(remove_spac)
movies['crew']=movies['crew'].apply(remove_spac)

In [ ]:

def remove_spaces_from_genres(data):
    """Removes spaces from genre names in the genres list."""
    if isinstance(data, list):
        return [genre.replace(" ", "") for genre in data]
    return data

# Apply the function to the 'genres' column
movies['genres'] = movies['genres'].apply(remove_spaces_from_genres)


In [ ]:
def remove_spaces_from_keywords(data):
    """Removes spaces from each keyword in the keywords list."""
    if isinstance(data, list):  # If it's a list of keywords
        return [keyword.replace(" ", "") for keyword in data]
    return data

# Apply the function to the 'keywords' column
movies['keywords'] = movies['keywords'].apply(remove_spaces_from_keywords)


In [ ]:
movies['tags']=movies['overview']+movies['genres'] +movies['cast'] + movies['crew'] + movies['keywords']

In [ ]:
# Create a new DataFrame with selected columns
new_df = movies[['id', 'title', 'tags']]

# Display the first 3 rows
new_df.head(3)


In [ ]:
print(new_df['tags'].head(10))  # Inspect the first few rows
print(new_df['tags'].apply(type).value_counts())  # Check for different types in th

In [ ]:
# Convert tags lists into a single string and handle NaN values
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x) if isinstance(x, list) else x)
new_df['tags'] = new_df['tags'].fillna("No Tags")  # Fill any remaining NaN values


In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

In [173]:
import nltk
from nltk.stem import  PorterStemmer

In [174]:
ps= PorterStemmer()

In [175]:
def stems(text):
    l=[]
    for i in text.split():
        l.append(ps.stem(i))
        return " ".join(l)

In [176]:
new_df['tags']=new_df['tags'].apply(stems)

C:\Users\Hp\AppData\Local\Temp\ipykernel_26232\1522715013.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stems)


In [179]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000, stop_words='english')


In [180]:
vector=cv.fit_transform(new_df['tags']).toarray()

In [181]:
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [182]:
vector.shape

(4806, 1184)

In [183]:
print("Number of unique features:", len(cv.get_feature_names_out()))


Number of unique features: 1184


In [184]:
X = cv.fit_transform(new_df['tags'])
print("Shape of document-term matrix:", X.shape)  # Check the number of features


Shape of document-term matrix: (4806, 1184)


In [185]:
cv = CountVectorizer(max_features=10000, stop_words='english')


In [186]:
X = cv.fit_transform(new_df['tags'])
print("Shape of document-term matrix:", X.shape)  # Check the number of features


Shape of document-term matrix: (4806, 1184)


In [187]:
print("Number of unique features:", len(cv.get_feature_names_out()))


Number of unique features: 1184


In [189]:
from sklearn.metrics.pairwise import cosine_similarity


In [193]:
similarity=cosine_similarity(vector).shape


In [194]:
similarity

(4806, 4806)

In [199]:
similarity_matrix[0][9]

np.float64(0.0)

In [196]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# Example data (using `new_df['tags']` here)
cv = CountVectorizer(max_features=5000, stop_words='english')
X = cv.fit_transform(new_df['tags'])

# Compute the cosine similarity matrix
similarity_matrix = cosine_similarity(X)

# Print the shape of the similarity matrix (should be (4806, 4806))
print(similarity_matrix.shape)

# Accessing the first row (should be a 1D array with 4806 elements)
print(similarity_matrix[0].shape)  # Should be (4806,)


(4806, 4806)
(4806,)
